<font color=darkred>

# Soc220: Computational Text Analysis
## Lab3: APIs and data storage options

<br>

<img src="tillerson.png" alt="Drawing" style="width: 600px;"/>

***
    2/7/2018
    Zach Wehrwein

<font color=darkpurple>

### Solutions

* 'class' in python is a default command use to program custom objects and methods, so that's why you use class_ in soup.

In [ ]:
import re
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pickle
import json
import os

#change directory


In [ ]:
!cd /Users/zwehrwein/Dropbox/Coursework/2018Spring/soc220_spring2018/soc220-labs-draft/lab3/

In [ ]:
#LIST OF SPEECH URLS


url = "http://www.presidency.ucsb.edu/sou.php"
req = requests.get(url,timeout=20) #always include a delay!
req.status_code 

In [ ]:
req.html = req.text
soup = BeautifulSoup(req.html,"html.parser")

<img src="nixon.png" alt="Drawing" style="width: 600px;"/>

### Need to get right URLS as a result (just in the table).

In [ ]:
#Get the right list of URLS
#VER12 gets us the main table and not the extra links
speech_url_list = []
for span in soup.find_all('td', class_="ver12"):
    #print(span)
    if len(span.find_all('a',href = re.compile('pid'))) > 0:
        link = span.find('a',href = re.compile('pid'))['href']
        speech_url_list.append(link)
        print(link)

In [ ]:
len(speech_url_list)

In [ ]:
#create a new directory to store data.
dir_path = "sotu_speeches"
if os.path.exists(dir_path) == False:
    os.makedirs(dir_path)

os.chdir('sotu_speeches')
   
#print working directory
!pwd

In [ ]:
titles_speeches = []
text_of_speeches = []

for url in speech_url_list:
    
    #10x timeout
    req = requests.get(url,timeout=200)
    req.html = req.text
    soup = BeautifulSoup(req.html,"html.parser")
    
    #date
    try:
        date = soup.find('span', {'class' : 'docdate'}).get_text()
        #dates_of_speeches.append(date)
        #print('processed:',len(dates_of_speeches),date)
    except:
        print('FAIL ON DATE:',len(dates_of_speeches))
        
    #name
    try:
        name = soup.title.text.split(':')[0]
        last_name = name.split()[-1]
        first_name = name.split()[0]
        #names_of_prezs.append(name)
        #print('processed:',len(names_of_prezs),last_name)
    except:
        print('FAIL ON NAME:')
        
    title = last_name+'_'+first_name+'_'+date
    titles_speeches.append(title)
    print(title)
    
    #text
    speech_pars = []
    for p in soup.find('span', class_='displaytext').find_all('p'):
        speech_pars.append(p.get_text())
        
    text_of_speeches.append(str(speech_pars))
    print('PROCESSED:',len(text_of_speeches),'th SOTU')
    
    #write_out too
    f = open(title,'w')
    f.write(str(speech_pars))
    f.close()

In [ ]:
print(len(text_of_speeches)); print(len(titles_speeches))

In [ ]:
sotu_dict = dict(zip(titles_speeches,text_of_speeches))
pickle.dump(sotu_dict,open("sotu_dict.p","wb"))

***
***

<font color=darkgreen>

## Today:

1. How to use APIs
2. Twitter API examples: <br>
    (a.) #metoo <br>
    (b.) #maga <br>
    (c.) @realDonaldTrump 
3. Data storage options (JSON, CSV)

***

<font color=darkgreen>

### APIs

- A set of protocols for two forms of software to communicate with one another.
- Rather than loading a webpage and looking for content, APIs allow us to directly ask another machine for information.
- Compared to web scraping, APIs are far more sane
- Most major tech firms provide APIs
- API  stands for 'application programming interface' and it is essentially a set of tools that allow you to communicate directly with a data server.
- So, this means that all the little tricks we discussed last week (finding the right bit of HTML code, using a regex etc) are unnecessary
- Moreover, since you are getting consistent streams of data, then you can save these data in a way that is human usable. Recall my warning last week: when scraping, you wish to have the smootest possible request->scrape->save code because if there is any little bump in the road, the whole program can crash.

#### Stream vs REST API

- Twitter has two types of APIs, one is the stream (a continuous unlimited flow of real time information), while the other is REST -- a repository which is one week old and is rate limited: 180 requests of up to 100 tweets per request every 15 minutes. (1200 tweets a minute)
- Twitter is bad about providing historical data, but this is not true of other firms. If you want Twitter data, you'll have to let it run for a week.

#### Lots of possible APIs:
- Yelp
- NYT
- Wikipedia
- JSTOR
- Spotify
- Open Movie Database
- and many more!

#### In general, here's what we'll do:
1. Go to the website in question, create a 'developer' account and be given a set of credentials.
2a. Go on Github and find the repository of the developer who wrote the API and look for a file like "sample.py" that provides an example way to access the API.
2b. Following 2a, install the developer  module and feed your credentials.
3. Make requests of the API for specific data you want. (Always include a delay so you don't get blocked!)
4. If there are restrictions on the API (can only access recent data or you have to pay for more), then we think through work-around.



https://developer.twitter.com/en/docs

https://apps.twitter.com/

<img src="twitter1.png" alt="Drawing" style="width: 600px;"/>

<img src="twitter2.png" alt="Drawing" style="width: 600px;"/>

<img src="twitter3.png" alt="Drawing" style="width: 500px;"/>


In [ ]:
!pip install tweepy

In [ ]:
!pip install twitter

***

<font color=darkgreen>
    
### PART1: Collect Tweets from the public 'garden house' using Twitter's own Python library.

In [ ]:
from twitter import Twitter, OAuth, TwitterStream
import json #Twitter outputs data in JSON format, which we will save locally.

In [ ]:
# API credentials
API_KEY = "GMCwaYz3FjycjSNamORMsfh6o"
API_SECRET = "b95joF79aiZfs9b1tSVUliCsJMwZJasBBklHQuSVjcyl3zWgs0"
ACCESS_TOKEN = "813490235099201536-mCd8VIMWdjpgVZh6D4SCIu9YdSjJ3D4"
ACCESS_SECRET = "6aBBdZEf6PnGPpLnXFVZT0Jlov3e9kJOQySCbB0ihrmq9"

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, API_KEY, API_SECRET)

In [ ]:
# Connect to Twitter Stream API

twitter_stream = TwitterStream(auth=oauth)

In [ ]:
# Get a sample of tweets

tweets_sample = twitter_stream.statuses.sample()

In [ ]:
list_sample_tweets = list()
tweet_count = 100
for tweet in tweets_sample:
    
    # Count down from 0 to collect 1000 public tweets
    tweet_count -= 1
    
    print(tweet)
    list_sample_tweets.append(tweet)
    
    # Stop this loop once we hit 0.
    if tweet_count <= 0:
        break

In [ ]:
type(list_sample_tweets[5])

# Designed to function like a dictionary object.

<font color=darkgreen>

- text: the text of the actual tweet;
- created_at: the date of creation;
- favorite_count: the number of favorites of this tweet;
- retweet_count: the number of retweets;
- lang: the language of the tweet;
- id: the tweet identifier;
- place: geo-location information;
- user: the profile of the author of the tweet.


In [ ]:
list_sample_tweets[60]

In [ ]:
list_sample_tweets[22]['entities']

<font color=darkgreen>

**This isn't that interesting though:**

- Single hashtag?
- Keep it local?

Again, consult the docs:

https://twitterapi.readthedocs.io/en/latest/

https://developer.twitter.com/en/docs

https://github.com/sixohsix/twitter

https://github.com/ideoforms/python-twitter-examples

In [ ]:
metoo_tweets_list = list()

tweet_count = 10
#call the twitter stream, then the statuses method to then filter
for tweet in twitter_stream.statuses.filter(track='#metoo',language='en'):
    
    # Count down from 0 to collect 1000 public tweets
    tweet_count -= 1
    
    
    #print text of tweet to see what it says
    print(tweet['text'])
    
    #save data out
    metoo_tweets_list.append(tweet)
    #json.dumps(metoo_tweets_list)
    
    json_doc = json.dumps(metoo_tweets_list)
    #outputFile.write(json_doc.encode('utf8') + '\n')
    
    #open up a json file, call it metoo tweets, write to it
    out_json = open('metoo_tweets.json','w')
    out_json.write(json_doc)
    out_json.close()
    
    # Stop this loop once we hit 0.
    if tweet_count <= 0:
        break

In [ ]:
len(metoo_tweets_list)

In [ ]:
maga_tweets_list = list()

tweet_count = 10
#call the twitter stream, then the statuses method to then filter
for tweet in twitter_stream.statuses.filter(track='#maga',language='en'):
    
    # Count down from 0 to collect 1000 public tweets
    tweet_count -= 1
    
    
    #print text of tweet to see what it says
    print(tweet['text'])
    
    #save data out
    maga_tweets_list.append(tweet)
    #json.dumps(metoo_tweets_list)
    
    json_doc = json.dumps(maga_tweets_list)
    #outputFile.write(json_doc.encode('utf8') + '\n')
    
    #open up a json file, call it metoo tweets, write to it
    out_json = open('maga_tweets.json','w')
    out_json.write(json_doc)
    out_json.close()
    
    # Stop this loop once we hit 0.
    if tweet_count <= 0:
        break

In [ ]:
len(metoo_tweets_list)

<font color=darkgreen>

### JSON files

- Use the new Firefox browser to open JSON files
- out of Python structured as a dictionary (that is, key-value pairs)
- json_x.load() will load the json object 'json_x' as a dictionary in Py
- Because they are saved locally, this requires some code to tell the computer what to save


`JSON_DUMP = json.dumps(ENTITY)` This takes a Python objects and puts into a json object that is now held in memory. <br>
`LOCAL_FILE_NAME = open('LOCAL_FILE_NAME'.json, 'w')` 'w' means 'write to file'; this opens a local file. <br>
`LOCAL_FILE_NAME.write(JSON_DUMP)` Take that json object you have in memory and write it to the local file. <br>
`LOCAL_FILE_NAME.close()` Close and save local file. n.b. the json object is still held in memory at this point, but it's also out of the Python program. <br>

Likewise, to open up a local file, do this: <br>
`
with open("LOCAL_FILE_NAME.json") as JSON_FILE:
    back_to_py = json.load(JSON_FILE)`

***

<font color=darkgreen>
    
### PART2: For more detailed searches (beyond just the stream), we'll turn to a wrapper around the Twitter API called tweepy

https://github.com/tweepy/tweepy

Docs: https://media.readthedocs.org/pdf/tweepy/v3.2.0/tweepy.pdf

**n.b tweepy returns a different type of object compared to the Twitter api -- a 'status' object which functions a bit differently -- always important to read the docs**

In [ ]:
import tweepy
from tweepy import OAuthHandler

In [ ]:
#AUTHENTICATE

auth = OAuthHandler(API_KEY,API_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

<font color=darkgreen>
    
* `wait_on_rate_limit=True` means that tweepy will exhaust the amount of 'calls' you can make to the API and wait for a refresh. In order to avoid being overwhelmed, most APIs specify a certain number of calls per hour (always check their documentation).

Full tweepy docs:

http://docs.tweepy.org/en/v3.5.0/getting_started.html


https://media.readthedocs.org/pdf/tweepy/latest/tweepy.pdf

Best tweepy practices:

http://www.nirg.net/using-tweepy.html

***

In [ ]:
# CHECK STATUS IS AN OPTION WITH tweepy
print(api.me().name)

<font color=darkgreen>

## CSV files

- 'comma seperated values'
- flexible and useful for almost all dataframe like analyses
- Not widely used because we're creating a data structure perhaps before we've really thought about it (flat frame)
- You have to specify which bits of data that you wish.

For 'status' objects created by tweepy, these are the following:

* 'author',
* 'contributors',
* 'coordinates',
* 'created_at',
* 'destroy',
* 'entities',
* 'extended_entities',
* 'favorite',
* 'favorite_count',
* 'favorited',
* 'geo',
* 'id',
* 'id_str',
* 'in_reply_to_screen_name',
* 'in_reply_to_status_id',
* 'in_reply_to_status_id_str',
* 'in_reply_to_user_id',
* 'in_reply_to_user_id_str',
* 'is_quote_status',
* 'lang',
* 'metadata',
* 'parse',
* 'parse_list',
* 'place',
* 'possibly_sensitive',
* 'retweet',
* 'retweet_count',
* 'retweeted',
* 'retweets',
* 'source',
* 'source_url',
* 'text',
* 'truncated',
* 'user'

<font color=darkgreen>

### PART3: getting a handle on the current status of American foreign policy (i.e. downloading @realDonaldTrump's timeline)

In [ ]:
all_trump_tweets = list()
all_trump_file = open('all_trump_tweets.csv', 'a')
all_trump_writer = csv.writer(all_trump_file)

#initial request
new_trump_tweets = api.user_timeline(screen_name='realDonaldTrump',count=200)
all_trump_tweets.extend(new_trump_tweets)
oldest_tweet = all_trump_tweets[-1].id - 1
oldest_created_at = all_trump_tweets[-1].created_at

while len(new_trump_tweets) > 0:
    print('Getting a new round of Trump tweets starting from:', oldest_created_at)
    
    new_trump_tweets = api.user_timeline(screen_name='realDonaldTrump',count=200,max_id=oldest_tweet)
    
    all_trump_tweets.extend(new_trump_tweets)
    
    #write out

    for tweet in new_trump_tweets:
        out_data = [tweet.created_at,
                    tweet.favorite_count,
                    tweet.in_reply_to_screen_name,
                    tweet.retweet_count,
                    tweet.source,
                    tweet.text.encode("utf-8")]
    
        all_trump_writer.writerow(out_data)
    
    oldest_tweet = all_trump_tweets[-1].id - 1
    oldest_created_at = all_trump_tweets[-1].created_at
    
    time.sleep(60)
    
    print('\n',len(all_trump_tweets),'Trump tweets have been collected so far.','\n')
    
    
all_trump_file.close()    

<font color=darkred>
    
### Our file is about 35,000 tweets short of getting the whole corpus -- what could we do?

<br>
<br>
<br>

-- Include delays? <br>
-- Call a larger iteration <br>
-- Change the dates for 'oldest' tweet? <br>
-- Except / Pass error messages. <br>
-- Does tweepy have a function to let you see your current call status? Could we build something off that? <br>
-- You could just keep running that cell over and over again... but why?

***

In [ ]:
api = tweepy.API(auth)

In [ ]:
api.rate_limit_status()['resources']['users']

In [ ]:
all_trump_tweets = list()
all_trump_file = open('all_trump_tweets.csv', 'a')
all_trump_writer = csv.writer(all_trump_file)

#initial request
new_trump_tweets = api.user_timeline(screen_name='realDonaldTrump',count=200)
all_trump_tweets.extend(new_trump_tweets)
oldest_tweet = all_trump_tweets[-1].id - 1
oldest_created_at = all_trump_tweets[-1].created_at

while len(new_trump_tweets) > 0:
    print('Getting a new round of Trump tweets starting from:', oldest_created_at)
    
    new_trump_tweets = api.user_timeline(screen_name='realDonaldTrump',count=200,max_id=oldest_tweet)
    
    all_trump_tweets.extend(new_trump_tweets)
    
    #write out

    for tweet in new_trump_tweets:
        out_data = [tweet.created_at,
                    tweet.favorite_count,
                    tweet.in_reply_to_screen_name,
                    tweet.retweet_count,
                    tweet.source,
                    tweet.text.encode("utf-8")]
    
        all_trump_writer.writerow(out_data)
    
    oldest_tweet = all_trump_tweets[-1].id - 1
    oldest_created_at = all_trump_tweets[-1].created_at
    
    ### INCLUDE A TIME DELAY###
    time.sleep(60)
    
    print('\n',len(all_trump_tweets),'Trump tweets have been collected so far.','\n')
    
    
all_trump_file.close()    

<font color=darkgreen>
    
### Use the RCE

https://rce-docs.hmdc.harvard.edu/book/accessing-rce-0

<font color=darkred>

## Exercise : 

#### Who does Donald Trump read? Can you write a simple Python scrip to detect when the nuke's gonna fly?

In other words, can you recreate Trump's feed in a CSV? Itterate through a list of his friends and call the 5 most recent tweets and save this to a CSV file. Then sort either in a notebook or in the CSV file itself by date to recreate his actual feed.

In [ ]:
trump_friends = []
for friend in api.get_user('realDonaldTrump').friends(count=50):
    trump_friends.append(friend.screen_name)

In [ ]:
trump_friends

<font color=green>

# TO RECAP:

* **APIs are better than scrapping HTML / XML**
* **There are data restriction limits however these will be app specific. For instance, Spotify will only give you named Playlists, so you have to find a way to get a large list of playlist names. With Twitter, telling the computer to keep trying after it has hit its limit.**
* **Look to Github and docs for examples.**